In [ ]:
#install required packages
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

     |████████████████████████████████| 7.9 MB 4.5 MB/s 
     |████████████████████████████████| 3.5 MB 4.0 MB/s 


In [ ]:
# helper function for visualization
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

def visualize(h, color):
  z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

  plt.figure(figsize = (10, 10))
  plt.xticks([])
  plt.yticks([])

  plt.scatter(z[:, 0], z[:, 1], s = 70, c = color, cmap = 'Set2')
  plt.show()

In [ ]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

def data(name):
  dataset = Planetoid(root = "data/Planetoid", name = f"{name}", transform = NormalizeFeatures())
  return dataset
dataset = data("Citeseer") # chnage dataset type
print(dataset) 

print("---"*36)
print(f'length of the dataset: {len(dataset)}')
print(f'features in the dataset: {dataset.num_features}')
print(f'Number of classes in the dataset: {dataset.num_classes}')
print("---"*36)
data = dataset[0]
print('print dataset properties')
print(data)

print(f'Number of nodes in the dataset: {data.num_nodes}')
print(f'Number of edges in the dataset: {data.num_edges}')
print(f'Average Node degree: {data.num_edges/data.num_nodes:0.2f}')
print(f'number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {data.train_mask.sum()/data.num_nodes:0.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Citeseer()
------------------------------------------------------------------------------------------------------------
length of the dataset: 1
features in the dataset: 3703
Number of classes in the dataset: 6
------------------------------------------------------------------------------------------------------------
print dataset properties
Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])
Number of nodes in the dataset: 3327
Number of edges in the dataset: 9104
Average Node degree: 2.74
number of training nodes: 120
Training node label rate: 0.04
Has isolated nodes: True
Has self-loops: False
Is undirected: True


In [ ]:
import torch
from torch.nn import Linear
import torch.nn.functional as F

class MLP(torch.nn.Module):
  def __init__(self, hidden_channles):
    super().__init__()
    torch.manual_seed(12345)
    self.l1 = Linear(dataset.num_features, hidden_channles)
    self.l2 = Linear(hidden_channles, dataset.num_classes)

  def forward(self, x):
    x = self.l1(x)
    x = F.relu(x)
    x = F.dropout(x, p = 0.5, training = self.training)
    x = self.l2(x)
    return x


model = MLP(hidden_channles=16)
print(model)


MLP(
  (l1): Linear(in_features=3703, out_features=16, bias=True)
  (l2): Linear(in_features=16, out_features=6, bias=True)
)


In [ ]:
model = MLP(hidden_channles=16)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01, weight_decay = 5e-4)


In [ ]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))


def train():
  model.train()
  optimizer.zero_grad()
  out = model(data.x)
  loss = criterion(out[data.train_mask], data.y[data.train_mask])
  loss.backward()
  optimizer.step()
  return loss

def test():
  model.eval()
  out = model(data.x)
  pred = out.argmax(dim=1)
  test_correct = (pred[data.test_mask] == data.y[data.test_mask])
  test_acc = int(test_correct.sum())/int(data.test_mask.sum())
  return test_acc
for epoch in range(1, 201):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Loss: 1.8032
Epoch: 002, Loss: 1.7984
Epoch: 003, Loss: 1.7924
Epoch: 004, Loss: 1.7849
Epoch: 005, Loss: 1.7753
Epoch: 006, Loss: 1.7695
Epoch: 007, Loss: 1.7571
Epoch: 008, Loss: 1.7482
Epoch: 009, Loss: 1.7426
Epoch: 010, Loss: 1.7279
Epoch: 011, Loss: 1.7184
Epoch: 012, Loss: 1.6971
Epoch: 013, Loss: 1.6868
Epoch: 014, Loss: 1.6798
Epoch: 015, Loss: 1.6684
Epoch: 016, Loss: 1.6549
Epoch: 017, Loss: 1.6277
Epoch: 018, Loss: 1.6293
Epoch: 019, Loss: 1.6114
Epoch: 020, Loss: 1.5793
Epoch: 021, Loss: 1.5792
Epoch: 022, Loss: 1.5556
Epoch: 023, Loss: 1.5405
Epoch: 024, Loss: 1.5248
Epoch: 025, Loss: 1.4971
Epoch: 026, Loss: 1.4818
Epoch: 027, Loss: 1.4754
Epoch: 028, Loss: 1.4292
Epoch: 029, Loss: 1.4271
Epoch: 030, Loss: 1.4126
Epoch: 031, Loss: 1.3647
Epoch: 032, Loss: 1.3686
Epoch: 033, Loss: 1.3339
Epoch: 034, Loss: 1.3454
Epoch: 035, Loss: 1.2883
Epoch: 036, Loss: 1.3052
Epoch: 037, Loss: 1.2462
Epoch: 038, Loss: 1.2758
Epoch: 039, Loss: 1.1842
Epoch: 040, Loss: 1.1490


In [ ]:
test_acc = test()
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.5820
